# vGWAS.jl

vGWAS.jl is a Julia package for performing genome-wide association studies (GWAS) for continuous longitudinal phenotypes using a modified linear mixed effects model. It builds upon the [within-subject variance estimation by robust regression (WiSER) model estimation](https://github.com/OpenMendel/WiSER.jl) and can be used to identify variants associated with changes in the mean and within-subject variability of the longitduinal trait. The estimation procedure is robust to both distributional misspecifications of the random effects and the response. A saddlepoint approximation (SPA) option is implemented in order to provide improved power and decreased type I error for rare variants. 


vGWAS.jl currently supports [PLINK](https://zzz.bwh.harvard.edu/plink/), [VCF](https://en.wikipedia.org/wiki/Variant_Call_Format) (both dosage and genotype data) file formats, and [BGEN](https://www.well.ox.ac.uk/~gav/bgen_format/) file formats. We plan to add [PGEN](https://www.cog-genomics.org/plink/2.0/formats#pgen) support in the future. 

## Installation

This package requires Julia v1.5 or later and four other unregistered packages SnpArrays.jl, VCFTools.jl, BGEN.jl, and WiSER.jl. The package has not yet been registered and must be installed using the repository location. Start julia and use the ] key to switch to the package manager REPL and run:
```julia
(@v1.5) pkg> add https://github.com/OpenMendel/SnpArrays.jl
(@v1.5) pkg> add https://github.com/OpenMendel/VCFTools.jl
(@v1.5) pkg> add https://github.com/OpenMendel/BGEN.jl
(@v1.5) pkg> add https://github.com/OpenMendel/WiSER.jl
(@v1.5) pkg> add https://github.com/OpenMendel/vGWAS.jl
```

In [1]:
# machine information for this tutorial
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i7-4850HQ CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
# for use in this tutorial
ENV["COLUMNS"] = 250
using BenchmarkTools, CSV, Glob, SnpArrays, vGWAS

┌ Info: Precompiling vGWAS [9514c204-b736-47c9-8157-11c3e9e5ab30]
└ @ Base loading.jl:1278


## Example data sets

The `data` folder of the package contains the example data sets for use with PLINK and VCF Files. In general, the user can locate this folder by command:

In [3]:
using vGWAS
pvalpath = "vgwas.pval.txt"
nullpath = "vgwas.null.txt"
const datadir = normpath(joinpath(dirname(pathof(vGWAS)), "../data/"))

"/Users/christophergerman/.julia/dev/vGWAS/data/"

In [4]:
# content of the data folder
readdir(glob"*.*", datadir)

12-element Array{String,1}:
 "/Users/christophergerman/.julia/dev/vGWAS/data/example.8bits.bgen"
 "/Users/christophergerman/.julia/dev/vGWAS/data/example.8bits.bgen.bgi"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.fam"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap_snpsetfile.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/sim_data.jl"
 "/Users/christophergerman/.julia/dev/vGWAS/data/snpsetfile_vcf.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/test_vcf.vcf.gz"
 "/Users/christophergerman/.julia/dev/vGWAS/data/vgwas_bgen_ex.csv"
 "/Users/christophergerman/.julia/dev/vGWAS/data/vgwas_plinkex.csv"
 "/Users/christophergerman/.julia/dev/vGWAS/data/vgwas_vcfex.csv"

The `hapmap3` files `vgwas_plinkex.csv` file correspond to data examples using PLINK formatted files (.bed, .bim, .fam). 

The `test_vcf.vcf.gz` and `vgwas_vcfex.csv` files are for an example analysis using VCF formatted files. 

The `example.8bits.bgen` and `vgwas_bgen_ex.csv` files are for an example analysis using VCF formatted files. 


## Basic usage

The following command performs GWAS using for the hapmap3 PLINK files. The output is the fitted null model.

The default type of GWAS performed is a single-snp significance genome-wide scan, this can be changed by the keyword `analysistype` (default is "singlesnp"). Other types of analyses are gone over later. It outputs the null model, runtime of fitting the null model, and convergence metrics. 

In [7]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = pvalpath,
        nullfile = nullpath)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.153256
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.140690



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



For documentation of the `vgwas` function, type `?vgwas` in Julia REPL.
```@docs
vgwas
```

### Formula for null model

The first three arguments specify the null model without SNP effects. The fourth argument specifies the grouping variable (subject ID, cluster, etc.).

- The first term is the mean fixed effects formula e.g., `@formula(y ~ 1 + sex + onMeds)`.
- The second argument is the random (location) effects -- `@formula(y ~ 1)` specifies a random intercept, whereas `@formula(y ~ 1 + time)` would specify a random intercept and slope (time).
- The third argument specifies the within-subject variance (WSV) formula `@formula(y ~ 1 + sex + onMeds)` models the intra-individual variability of `y` with an intercept, sex, and medication. Note: The WSV formula must have an intercept. 
- The fourth argument specifies the grouping variable of the longitudinal data. In the example data, the `id` variable specifies the grouping variable the repeated measures are collected on. 



### Input files

`vgwas` expects two input files: one for responses plus covariates (fifth argument), the other the genetic file(s) for dosages/genotypes (sixth argument).

#### Covariate and trait file

Covariates and phenotype are provided in a csv file, e.g., `vgwas_plinkex.csv`, which has one header line for variable names. In this example, variable `y` is continuous variable with repeated measures on individuals specified by `id`. We want to include variable `sex` and `onMeds` as covariates for GWAS.

In [19]:
run(`head $(datadir)vgwas_plinkex.csv`);

sex,onMeds,snp1,snp2,snp3,snp4,y,id
0.0,1.0,0.0,1.0,2.0,0.0,12.26667411332518,A1
0.0,0.0,0.0,1.0,2.0,0.0,10.268123812744903,A1
0.0,0.0,0.0,1.0,2.0,0.0,12.165997408822557,A1
0.0,0.0,0.0,1.0,2.0,0.0,11.879709602937222,A1
0.0,0.0,0.0,1.0,2.0,0.0,12.812705165990007,A1
0.0,0.0,0.0,1.0,2.0,0.0,9.987659617201372,A1
0.0,0.0,0.0,1.0,2.0,0.0,12.140779426464974,A1
0.0,1.0,0.0,1.0,2.0,0.0,13.205778146177705,A1
0.0,0.0,0.0,1.0,2.0,0.0,11.363145919060207,A1


#### Genetic file(s)

vGWAS supports PLINK files, VCF files, and BGEN files.

- Genotype data is available as binary PLINK files.

- vGWAS can use dosage or genotype data from VCF files. 

- vGWAS uses dosage data from BGEN files.

!!! note

    By default, vGWAS assumes a set of PLINK files will be used. When using a VCF File, VCF file and type of data (dosage, genotype) must be specified by the `geneticformat` and `vcftype` options (as shown later). Similarly, BGEN must be specified as the `geneticformat` if using a BGEN file. 

In [11]:
readdir(glob"hapmap3.*", datadir)

3-element Array{String,1}:
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.fam"

In this example, there are 324 samples at 13,928 SNPs.

In [12]:
size(SnpArray(datadir * "hapmap3.bed"))

(324, 13928)

Compressed PLINK and VCF files are supported. For example, if Plink files are `hapmap3.bed.gz`, `hapmap3.bim.gz` and `hapmap3.fam.gz`, the same command
```julia
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = pvalpath,
        nullfile = nullpath)
```
still works. Check all supported compression format by

In [11]:
SnpArrays.ALLOWED_FORMAT

6-element Array{String,1}:
 "gz"
 "zlib"
 "zz"
 "xz"
 "zst"
 "bz2"

### Output files

`vgwas` outputs two files: `vgwas.null.txt` and `vgwas.pval.txt`. 

* `vgwas.null.txt` lists the estimated null model (without SNPs). 

In [13]:
run(`cat vgwas.null.txt`);


Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



* `vgwas.pval.txt` tallies the SNPs, their pvalues, and relevant information on each SNP.
    - betapval represents the p-value of the SNP's effect on the mean of the trait. If `spa=true` (default), then this is the SPA p-value. If `spa=false`, then this is the score test beta p-value. 
    - taupval represents the p-value of the SNP's effect on the within-subject variability of the trait. If `spa=true` (default), then this is the SPA p-value. If `spa=false`, then this is the score test tau p-value. 
    - jointpval represents a joint p-value of the SNP's effect on both the mean and variance. By default `spa=true` this is the harmonic mean of the saddlepoint approximated p-values for beta and tau. If `spa=false`, this is the joint score test p-value.

In [8]:
first(CSV.read("vgwas.pval.txt", DataFrame), 8)

,chr,pos,snpid,maf,hwepval,betapval,taupval,jointpval
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64
1,1,554484,rs10458597,0.0,1.0,1.0,1.0,1.0
2,1,758311,rs12562034,0.0776398,0.409876,0.38939,0.922822,0.689845
3,1,967643,rs2710875,0.324074,4.07625e-7,5.83856e-6,3.35408e-6,1.1501e-6
4,1,1168108,rs11260566,0.191589,0.128568,0.000850889,0.0559055,0.000739637
5,1,1375074,rs1312568,0.441358,2.5376e-19,0.000171683,1.84811e-13,4.62603e-14
6,1,1588771,rs35154105,0.0,1.0,1.0,1.0,1.0
7,1,1789051,rs16824508,0.00462963,0.933278,0.295035,0.304109,0.260174
8,1,1990452,rs2678939,0.453704,5.07696e-11,1.27871e-7,7.73809e-9,1.56973e-9


Output file names can be changed by the `nullfile` and `pvalfile` keywords respectively. For example, 
```julia
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "vgwas.pval.txt.gz",
        nullfile = nullpath)

```
will output the p-value file in compressed gz format.

### Subsamples

Use the keyword `covrowinds` to specify selected samples in the covarite file. Use the keyword `geneticrowinds` to specify selected samples in the Plink (.bed), VCF, or BGEN file. For example, to use the first 300 samples in both covariate and bed file:
```julia
vgwas(@formula(trait ~ 1+ sex), 
    @formula(trait ~ 1), 
    @formula(trait ~ 1 + sex), 
    :id,
    covfile, geneticfile, 
    covrowinds=1:300, geneticrowinds=1:300)
```
!!! note

    Users should make sure that the selected samples in covariate file match exactly those in bed file, otherwise an error will be displayed. 

### Input non-genetic data as DataFrame

Internally `vgwas` parses the covariate file as a DataFrame by `df = CSV.read(covfile, DataFrame)`. For covariate file of other formats, or for specifying default levels of categorical variables, users can parse their datafile as a DataFrame and then input the DataFrame to `vgwas` directly.
```julia
vgwas(@formula(trait ~ 1+ sex), 
    @formula(trait ~ 1), 
    @formula(trait ~ 1 + sex),
    :id,
    df, geneticfile)
```
!!! note

    Users should always make sure that individuals in covariate file or DataFrame match those in Plink fam file/VCF File/BGEN file. 

For example, following code checks that the first 2 columns of the `covariate.txt` file match the first 2 columns of the `hapmap3.fam` file exactly.

In [19]:
covdf = CSV.read(datadir * "covariate.txt")
plkfam = CSV.read(datadir * "hapmap3.fam", header=0, delim=' ')
all(covdf[!, 1] .== plkfam[!, 1]) && all(covdf[!, 2] .== plkfam[!, 2])

true

### Timing

For this moderate-sized data set, `vgwas` takes around 1 second without applying SPA. With SPA, it takes a little longer. 

In [23]:
@btime(vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = pvalpath,
        nullfile = nullpath, 
        usespa = false));

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.159280
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.149766
run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.152873
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.126264
run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.149566
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.130830
run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.141506
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.127524
run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.148520
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.130201
run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖

In [76]:
# clean up
rm("vgwas.null.txt", force=true)
rm("vgwas.pval.txt", force=true)

## VCF Formatted Files

By default, vGWAS.jl will assume you are using PLINK files. It also supports VCF (and BGEN) Files. `vcf.gz` files are supported as well. To use vcf files in any of the analysis options detailed in this documentation, you simply need to add two keyword options to the `vgwas` function:
* `geneticformat`: Choices are "VCF" or "PLINK". If you are using a VCF file, use `geneticformat = "VCF"`.
* `vcftype`: Choices are :GT (for genotypes) or :DS (for dosages). This tells vGWAS which type of data to extract from the VCF file.

Using a VCF File does not output minor allele frequency or hardy weinberg equillibrium p-values for each SNP tested since they may be dosages. 

The following shows how to run an analysis with a VCF file using the dosage information. 

In [72]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_vcfex.csv",
        datadir * "test_vcf",
        pvalfile = pvalpath,
        geneticformat = "VCF",
        vcftype = :DS)

run = 1, ‖Δβ‖ = 0.003459, ‖Δτ‖ = 0.421747, ‖ΔL‖ = 0.002492, status = Optimal, time(s) = 0.067161
run = 2, ‖Δβ‖ = 0.001369, ‖Δτ‖ = 0.015998, ‖ΔL‖ = 0.003244, status = Optimal, time(s) = 0.057506



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 191
Total observations: 1910

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  12.1708     0.18351     66.32    <1e-99
β2: sex          -3.41012    0.253155   -13.47    <1e-40
β3: onMeds        0.485447   0.0701423    6.92    <1e-11
τ1: (Intercept)   0.774299   0.0849867    9.11    <1e-19
τ2: sex          -0.524117   0.106719    -4.91    <1e-6
τ3: onMeds        0.633337   0.0814974    7.77    <1e-14
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  2.86452



In [73]:
first(CSV.read("vgwas.pval.txt", DataFrame), 8)

,chr,pos,snpid,betapval,taupval,jointpval
,Int64,Int64,String,Float64,Float64,Float64
1,22,20000086,rs138720731,0.322684,0.61956,0.593855
2,22,20000146,rs73387790,1.0,1.0,1.0
3,22,20000199,rs183293480,0.214207,0.190785,0.424942
4,22,20000291,rs185807825,0.292231,0.200029,0.387984
5,22,20000428,rs55902548,0.0140114,0.00369005,0.0059257
6,22,20000683,rs142720028,1.0,1.0,1.0
7,22,20000771,rs114690707,0.536406,0.145113,0.239854
8,22,20000793,rs189842693,0.161414,0.757017,0.271893


## BGEN Formatted Files

By default, vGWAS.jl will assume you are using PLINK files. It also supports BGEN Files. To use BGEN files in any of the analysis options detailed in this documentation, you simply need to add the following keyword option to the `vgwas` function:
* `geneticformat`: Choices are "VCF" or "PLINK" or "BGEN". If you are using a BGEN file, use `geneticformat = "BGEN"`.

Using a BGEN File does not output minor allele frequency or hardy weinberg equillibrium p-values for each SNP tested.

Some features, such as SNP-set analyses, are only available when there's an indexing `.bgi` file available. Additionally, if the BGEN file does not contain sample information (i.e. sample IDs), then a sample file is necessary and its path can be specified with the `samplepath` keyword. 

!!! note

    BGEN files can contain an optional index file (`.bgi` file) that allows the variants to be specified in order of position. vGWAS will automatically look for a file in the same directory as the `BGENFILENAME` with the name `BGENFILENAME.bgi`. The BGEN file is read either in the `.bgi` file order if `BGENFILENAME.bgi` is supplied in the same directory as the BGEN file, otherwise it will use the order in the BGEN file. This is important in analyses specifying `snpinds` as well as annotation groupings. You must make sure this matches the way the BGEN file will be read in. 

The following shows how to run an analysis with a BGEN file using the dosage information. 

In [5]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id, 
        datadir * "vgwas_bgen_ex.csv", 
        datadir * "example.8bits", 
        geneticformat = "BGEN", 
        pvalfile = pvalpath)

run = 1, ‖Δβ‖ = 0.096578, ‖Δτ‖ = 0.162563, ‖ΔL‖ = 0.007625, status = Optimal, time(s) = 0.195432
run = 2, ‖Δβ‖ = 0.003173, ‖Δτ‖ = 0.005584, ‖ΔL‖ = 0.001468, status = Optimal, time(s) = 0.145392



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 500
Total observations: 5000

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  10.5275     0.0992675  106.05    <1e-99
β2: sex          -3.38067    0.146416   -23.09    <1e-99
β3: onMeds        0.522162   0.0358727   14.56    <1e-47
τ1: (Intercept)   0.294364   0.0443191    6.64    <1e-10
τ2: sex          -0.365009   0.0503589   -7.25    <1e-12
τ3: onMeds        0.559769   0.0467046   11.99    <1e-32
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  2.51561



In [6]:
first(CSV.read("vgwas.pval.txt", DataFrame), 8)

,chr,pos,snpid,varid,betapval,taupval,jointpval
,Int64,Int64,String,String,Float64,Float64,Float64
1,1,1001,RSID_101,SNPID_101,0.510629,0.425112,0.601078
2,1,2000,RSID_2,SNPID_2,4.61579e-10,1.52897e-21,1.70132e-24
3,1,2001,RSID_102,SNPID_102,4.49315e-10,1.8004e-21,2.00297e-24
4,1,3000,RSID_3,SNPID_3,0.0721822,0.600949,0.188409
5,1,3001,RSID_103,SNPID_103,0.0721821,0.600949,0.188409
6,1,4000,RSID_4,SNPID_4,0.112472,0.133752,0.078067
7,1,4001,RSID_104,SNPID_104,0.112471,0.133752,0.078067
8,1,5000,RSID_5,SNPID_5,0.526325,0.542332,0.675825


## SNP models

Genotypes are translated into numeric values according to different genetic model, which is specified by the `snpmodel` keyword. Default is `ADDITIVE_MODEL`.

| Genotype | `SnpArray` | `ADDITIVE_MODEL` | `DOMINANT_MODEL` | `RECESSIVE_MODEL` |    
|:---:|:---:|:---:|:---:|:---:|  
| A1,A1 | 0x00 | 0 | 0 | 0 |  
| missing | 0x01 | NaN | NaN | NaN |
| A1,A2 | 0x02 | 1 | 1 | 0 |  
| A2,A2 | 0x03 | 2 | 1 | 1 |  

!!! note

    `vgwas` imputes missing genotypes according to minor allele frequencies. 
    
Users are advised to impute genotypes using more sophiscated methods before GWAS.

## SNP and/or sample masks

In practice, we often perform GWAS on selected SNPs and/or selected samples. They can be specified by the `snpinds`, `covrowinds` and `geneticrowinds` keywords of `vGWAS` function. 

For example, to perform GWAS on SNPs with minor allele frequency (MAF) above 0.05

In [8]:
# create SNP mask
snpinds = maf(SnpArray("../data/hapmap3.bed")) .≥ 0.05
# GWAS on selected SNPs
@time vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "commonvariant.pval.txt",
        snpinds = snpinds)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.194672
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.155585
 14.356801 seconds (28.98 M allocations: 1.466 GiB, 5.11% gc time)



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [9]:
run(`head commonvariant.pval.txt`);

chr	pos	snpid	maf	hwepval	betapval	taupval	jointpval
1	758311	rs12562034	0.07763975155279501	0.4098763332666681	0.38938987159564675	0.9228220976232299	0.689844925654112
1	967643	rs2710875	0.32407407407407407	4.076249100705747e-7	5.838562452891218e-6	3.354075049151564e-6	1.150102626573041e-6
1	1168108	rs11260566	0.19158878504672894	0.1285682279446898	0.000850889425950933	0.05590549977468809	0.0007396374017787817
1	1375074	rs1312568	0.441358024691358	2.5376019650614977e-19	0.00017168324418138272	1.8481146312335492e-13	4.626028588457252e-14
1	1990452	rs2678939	0.4537037037037037	5.07695957708431e-11	1.2787095295892262e-7	7.738089147582373e-9	1.5697329864658462e-9
1	2194615	rs7553178	0.22685185185185186	0.17056143157457776	0.07404001756947043	0.00023109048716815853	0.00015039840819071202
1	2396747	rs13376356	0.1448598130841121	0.9053079215078139	0.4897208868010412	0.6595593862223351	0.7352162221637434
1	2823603	rs1563468	0.4830246913580247	4.23065537243926e-9	2.581476982676445e-7	1.1700736

In [10]:
# extra header line in commonvariant.pval.txt
countlines("commonvariant.pval.txt"), count(snpinds)

(12086, 12085)

In [15]:
# clean up
rm("vgwas.null.txt", force=true)
rm("commonvariant.pval.txt", force=true)

`covrowinds` specify the samples in the covariate file and `geneticrowinds` for PLINK or VCF File. User should be particularly careful when using these two keyword. Selected rows in SnpArray should exactly match the samples in the null model. Otherwise the results are meaningless.

## Estimating Effect Sizes (Wald)

By default, `vgwas` calculates p-value for each SNP using SPA/score test. Score test is fast because it doesn't require fitting alternative model for each SNP. User can request the Wald p-values and the estimated effect size of each SNP using keyword `test=:wald`. Wald is much slower but will give you estimated effect sizes from the WiSER model.

In [19]:
@time vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "wald.pval.txt",
        snpinds = 1:5,
        test = :wald)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.161760
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.135861
run = 1, ‖Δβ‖ = 0.033243, ‖Δτ‖ = 0.146511, ‖ΔL‖ = 0.005476, status = Optimal, time(s) = 0.206452
run = 2, ‖Δβ‖ = 0.005774, ‖Δτ‖ = 0.042246, ‖ΔL‖ = 0.001784, status = Optimal, time(s) = 0.192730
run = 1, ‖Δβ‖ = 0.013090, ‖Δτ‖ = 0.130781, ‖ΔL‖ = 0.005011, status = Optimal, time(s) = 0.214504
run = 2, ‖Δβ‖ = 0.003913, ‖Δτ‖ = 0.037309, ‖ΔL‖ = 0.001516, status = Optimal, time(s) = 0.193909
run = 1, ‖Δβ‖ = 0.022159, ‖Δτ‖ = 0.141135, ‖ΔL‖ = 0.005554, status = Optimal, time(s) = 0.181490
run = 2, ‖Δβ‖ = 0.001482, ‖Δτ‖ = 0.021700, ‖ΔL‖ = 0.001435, status = Optimal, time(s) = 0.166887
run = 1, ‖Δβ‖ = 0.026764, ‖Δτ‖ = 0.368620, ‖ΔL‖ = 0.000317, status = Optimal, time(s) = 0.166970
run = 2, ‖Δβ‖ = 0.003023, ‖Δτ‖ = 0.030938, ‖ΔL‖ = 0.003568, status = Optimal, time(s) = 0.169333
  2.035903 seconds (456.79 k a


Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



Note the extra `effect` column in pvalfile, which is the effect size (regression coefficient) for each SNP. 

In [30]:
run(`head wald.pval.txt`);

chr,pos,snpid,maf,hwepval,effect,pval
1,554484,rs10458597,0.0,1.0,0.0,1.0
1,758311,rs12562034,0.07763975155279501,0.4098763332666681,-1.005783371954433,0.0019185836579805327
1,967643,rs2710875,0.32407407407407407,4.076249100705747e-7,-0.648856056629187,1.805050556976241e-5
1,1168108,rs11260566,0.19158878504672894,0.1285682279446898,-0.9157225669357901,5.8733847126869666e-6
1,1375074,rs1312568,0.441358024691358,2.5376019650614977e-19,-0.3318136652577225,0.008081022577828709
1,1588771,rs35154105,0.0,1.0,0.0,1.0
1,1789051,rs16824508,0.00462962962962965,0.9332783156468178,-0.7338026388700143,0.5169027130145593
1,1990452,rs2678939,0.4537037037037037,5.07695957708431e-11,-0.1358649923181975,0.2994640220091515
1,2194615,rs7553178,0.22685185185185186,0.17056143157457776,-0.2512075640440123,0.1615106909444108


In [20]:
# clean up
rm("wald.pval.txt", force=true)
rm("vgwas.null.txt", force=true)

## Score test for screening, Wald for effect size estimates 

For large data sets, a practical solution is to perform the score test first across all SNPs, then re-do Wald for the most promising SNPs according to score test p-values in order to get estimated effect sizes.

**Step 1**: Perform score test GWAS, results in `score.pval.txt`.

In [70]:
@time vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "score.pval.txt")

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.176625
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.138581
  2.666877 seconds (7.11 M allocations: 461.688 MiB, 4.68% gc time)



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [71]:
first(CSV.read("score.pval.txt", DataFrame), 8)

,chr,pos,snpid,maf,hwepval,betapval,taupval,jointpval
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64
1,1,554484,rs10458597,0.0,1.0,1.0,1.0,1.0
2,1,758311,rs12562034,0.0776398,0.409876,0.38939,0.922822,0.689845
3,1,967643,rs2710875,0.324074,4.07625e-7,5.83856e-6,3.35408e-6,1.1501e-6
4,1,1168108,rs11260566,0.191589,0.128568,0.000850889,0.0559055,0.000739637
5,1,1375074,rs1312568,0.441358,2.5376e-19,0.000171683,1.84811e-13,4.62603e-14
6,1,1588771,rs35154105,0.0,1.0,1.0,1.0,1.0
7,1,1789051,rs16824508,0.00462963,0.933278,0.295035,0.304109,0.260174
8,1,1990452,rs2678939,0.453704,5.07696e-11,1.27871e-7,7.73809e-9,1.56973e-9


**Step 2**: Sort score test p-values and find top 10 SNPs.

In [23]:
scorepvals = CSV.read("score.pval.txt", DataFrame)[!, :taupval] # tau p-values 
tophits = sortperm(scorepvals)[1:10] # indices of 10 SNPs with smallest p-values
scorepvals[tophits] # smallest 10 p-values

10-element Array{Float64,1}:
 7.872202557706978e-17
 1.8481146312335492e-13
 1.4978681413694476e-12
 9.730062616490623e-12
 1.2892059340654512e-11
 1.484463866022607e-11
 1.5167114834216833e-11
 1.876125621826194e-11
 2.349135295865958e-11
 2.6075629334507774e-11

**Step 3**: Re-do LRT on top hits.

In [33]:
@time vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "wald.pval.txt",
        snpinds = tophits,
        test = :wald)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.185237
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.141060
run = 1, ‖Δβ‖ = 0.026764, ‖Δτ‖ = 0.368620, ‖ΔL‖ = 0.000317, status = Optimal, time(s) = 0.205915
run = 2, ‖Δβ‖ = 0.003023, ‖Δτ‖ = 0.030938, ‖ΔL‖ = 0.003568, status = Optimal, time(s) = 0.166922
run = 1, ‖Δβ‖ = 0.039946, ‖Δτ‖ = 0.185049, ‖ΔL‖ = 0.005690, status = Optimal, time(s) = 0.179834
run = 2, ‖Δβ‖ = 0.001640, ‖Δτ‖ = 0.029639, ‖ΔL‖ = 0.002028, status = Optimal, time(s) = 0.172222
run = 1, ‖Δβ‖ = 0.040207, ‖Δτ‖ = 0.407863, ‖ΔL‖ = 0.001924, status = Optimal, time(s) = 0.172358
run = 2, ‖Δβ‖ = 0.002632, ‖Δτ‖ = 0.039609, ‖ΔL‖ = 0.005131, status = Optimal, time(s) = 0.176678
run = 1, ‖Δβ‖ = 0.052372, ‖Δτ‖ = 0.828946, ‖ΔL‖ = 0.001277, status = Optimal, time(s) = 0.214144
run = 2, ‖Δβ‖ = 0.013694, ‖Δτ‖ = 0.185493, ‖ΔL‖ = 0.007118, status = Optimal, time(s) = 0.182617
run = 1, ‖Δβ‖ = 0.032435, ‖Δτ‖


Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [34]:
CSV.read("wald.pval.txt", DataFrame)

,chr,pos,snpid,maf,hwepval,betaeffect,betapval,taueffect,taupval
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1375074,rs1312568,0.441358,2.5376e-19,-0.456032,0.000165921,0.496452,1.17735e-36
2,1,11552817,rs2745282,0.421053,1.3537e-14,-0.67282,7.6492e-9,0.425653,1.45395e-19
3,1,120276030,rs6688004,0.469136,1.75473e-27,-0.637817,5.13062e-9,0.430562,6.07457e-21
4,2,135623558,rs6730157,0.313272,7.76753e-16,-0.556166,1.10163e-5,0.462346,2.88628e-20
5,9,126307510,rs3814134,0.427469,4.28276e-33,0.723472,1.41601e-11,-0.518216,4.75338e-39
6,15,40603025,rs2617236,0.390966,1.16466e-14,-0.644375,1.44948e-7,0.445647,9.43451e-19
7,15,40803767,rs3742988,0.42284,4.26181e-30,-0.796743,2.24785e-13,0.389234,7.09349e-17
8,17,56509992,rs8064681,0.481481,1.37485e-21,-0.58961,1.9256e-7,0.425417,5.60171e-25
9,17,71293786,rs2125345,0.339009,1.75527e-14,-0.635669,1.82739e-7,0.449612,1.7146e-21


In [39]:
# clean up
rm("vgwas.null.txt", force=true)
rm("vgwas.pval.txt", force=true)
rm("score.pval.txt", force=true)
rm("wald.pval.txt", force=true)

## GxE or other interactions

### Testing jointly G + GxE 

In many applications, we want to test SNP effect and/or its interaction with other terms. `testformula` keyword specifies the test unit **besides** the covariates in `nullformula`. 

In following example, keyword `testformula=@formula(trait ~ snp + snp & sex)` instructs `vgwas` to test joint effect of `snp` and `snp & sex` interaction.

In [35]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "GxE.pval.txt",
        testformula=@formula(trait ~ snp + snp & sex))

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.149586
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.134007



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [38]:
first(CSV.read("GxE.pval.txt", DataFrame), 5)

,chr,pos,snpid,maf,hwepval,betapval,taupval,jointpval
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64
1,1,554484,rs10458597,0.0,1.0,1.0,1.0,1.0
2,1,758311,rs12562034,0.0776398,0.409876,0.383,0.33511,0.473632
3,1,967643,rs2710875,0.324074,4.07625e-7,9.98217e-6,5.73202e-7,9.96764e-11
4,1,1168108,rs11260566,0.191589,0.128568,0.00225789,0.105482,0.00342268
5,1,1375074,rs1312568,0.441358,2.5376e-19,0.000928728,3.48807e-15,7.51335e-16


In [41]:
# clean up
rm("vgwas.null.txt", force=true)
rm("GxE.pval.txt",  force=true)

### Testing only GxE interaction term

For some applications, the user may want to simply test the GxE interaction effect. This requires fitting the SNP in the null model and is much slower, but the command `vgwas()` with keyword `analysistype = "gxe"` can be used test the interaction effect.
The environmental variable must be specified in the command using the keyword argument `e`, either as a symbol, such as `:age` or as a string `"age"`. 

In [43]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "gxe_snp.pval.txt", 
        analysistype = "gxe",
        e = :sex, 
        snpinds=1:5)


run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.156353
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.152398
run = 1, ‖Δβ‖ = 0.033243, ‖Δτ‖ = 0.146511, ‖ΔL‖ = 0.005476, status = Optimal, time(s) = 0.192186
run = 2, ‖Δβ‖ = 0.005774, ‖Δτ‖ = 0.042246, ‖ΔL‖ = 0.001784, status = Optimal, time(s) = 0.178301
run = 1, ‖Δβ‖ = 0.013090, ‖Δτ‖ = 0.130781, ‖ΔL‖ = 0.005011, status = Optimal, time(s) = 0.214996
run = 2, ‖Δβ‖ = 0.003913, ‖Δτ‖ = 0.037309, ‖ΔL‖ = 0.001516, status = Optimal, time(s) = 0.208225
run = 1, ‖Δβ‖ = 0.022159, ‖Δτ‖ = 0.141135, ‖ΔL‖ = 0.005554, status = Optimal, time(s) = 0.177371
run = 2, ‖Δβ‖ = 0.001482, ‖Δτ‖ = 0.021700, ‖ΔL‖ = 0.001435, status = Optimal, time(s) = 0.155733
run = 1, ‖Δβ‖ = 0.026764, ‖Δτ‖ = 0.368620, ‖ΔL‖ = 0.000317, status = Optimal, time(s) = 0.171832
run = 2, ‖Δβ‖ = 0.003023, ‖Δτ‖ = 0.030938, ‖ΔL‖ = 0.003568, status = Optimal, time(s) = 0.161345



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [44]:
CSV.read("gxe_snp.pval.txt", DataFrame)

,chr,pos,snpid,maf,hwepval,snpeffectnullbeta,snpeffectnulltau,betapval,taupval,jointpval
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,554484,rs10458597,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,1,758311,rs12562034,0.0776398,0.409876,-0.231072,0.0134058,0.449093,0.147252,0.272065
3,1,967643,rs2710875,0.324074,4.07625e-7,0.640695,-0.347633,0.295171,0.12668,0.138741
4,1,1168108,rs11260566,0.191589,0.128568,0.618825,-0.147484,0.590334,0.270161,0.48269
5,1,1375074,rs1312568,0.441358,2.5376e-19,-0.456032,0.496452,0.574465,0.251019,0.463303


In [46]:
# clean up
rm("vgwas.null.txt", force=true)
rm("gxe_snp.pval.txt", force=true)

## SNP-set testing

In many applications, we want to test a SNP-set. The function with keyword `analysistype = "snpset"` can be used to do this. To specify the type of snpset test, use the `snpset` keyword argument. 

The snpset can be specified as either:
- a window (test every X snps) => `snpset = X`
- an annotated file.  This requires `snpset = filename` where filename is an input file, with no header and two columns separated by a space. The first column must contain the snpset ID and the second column must contain the snpid's (rsid) identical to the bimfile, or in the case of a BGEN format, the order the data will be read (see BGEN above for more details).
- a joint test on only a specific set of snps. `snpset = AbstractVector` where the vector specifies the snps you want to perform one joint snpset test for. The vector can either be a vector of integers where the elements are the indicies of the SNPs to test, a vector of booleans, where true represents that you want to select that SNP index, or a range indicating the indicies of the SNPs to test. 

SNPset testing is currently only implemented for the score test (not Wald). 

In the following example, we perform a SNP-set test on the 50th to 55th snps. 

In [47]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "snpset.pval.txt", 
        analysistype = "snpset",
        snpset = 50:55)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.153645
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.142342



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [48]:
run(`head snpset.pval.txt`);

The joint pvalue of snps indexed at 50:55 is betapval: 2.3156171211970728e-5, taupval: 3.972993706372347e-10, jointpval: 3.987431210288368e-12


In [49]:
# clean up
rm("snpset.pval.txt", force=true)
rm("vgwas.null.txt", force=true)

In the following example we run a SNP-set test on the annotated SNP-set file.

In [50]:
run(`head ../data/hapmap_snpsetfile.txt`);

gene1 rs10458597
gene1 rs12562034
gene1 rs2710875
gene1 rs11260566
gene1 rs1312568
gene1 rs35154105
gene1 rs16824508
gene1 rs2678939
gene1 rs7553178
gene1 rs13376356


In [51]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "snpset.pval.txt", 
        analysistype = "snpset",
        snpset = datadir * "/hapmap_snpsetfile.txt")

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.164930
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.146663



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [55]:
first(CSV.read("snpset.pval.txt", DataFrame), 5)

,snpsetid,nsnps,betapval,taupval,jointpval
,String,Int64,Float64,Float64,Float64
1,gene1,93,0.111864,0.011595,0.0536262
2,gene2,93,0.0249929,0.0648265,0.00863506
3,gene3,93,0.131741,0.0298884,0.0176667
4,gene4,92,0.010327,0.0584591,0.0126325
5,gene5,93,0.0303905,0.0924954,0.0494038


In [56]:
# clean up
rm("snpset.pval.txt", force=true)
rm("vgwas.null.txt", force=true)

In the following example we run a SNP-set test on every 15 SNPs.

In [57]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = "snpset.pval.txt", 
        analysistype = "snpset",
        snpset = 15)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.174487
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.156026



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [58]:
first(CSV.read("snpset.pval.txt", DataFrame), 5)

,startchr,startpos,startsnpid,endchr,endpos,endsnpid,betapval,taupval,jointpval
,Int64,Int64,String,Int64,Int64,String,Float64,Float64,Float64
1,1,554484,rs10458597,1,3431124,rs12093117,3.83046e-6,2.24303e-10,1.25591e-12
2,1,3633945,rs10910017,1,6514524,rs932112,0.000127922,7.94764e-6,5.98702e-8
3,1,6715827,rs441515,1,9534606,rs4926480,8.10353e-5,6.11848e-7,8.1905e-9
4,1,9737551,rs12047054,1,12559747,rs4845907,0.000425108,1.15311e-8,1.93616e-10
5,1,12760427,rs848577,1,16021797,rs6679870,2.75832e-5,0.00028553,9.27611e-7


In [60]:
# clean up
rm("snpset.pval.txt", force=true)
rm("vgwas.null.txt", force=true)

## Matching Indicies

In some cases, there are only a subset of individuals with both genetic data and covariate information available. The null model must be fit on a subset of the individuals with the genetic data. The rows can be specified with the argument `covrowinds` if you pass in a covariate file. The genetic indicies can be specified with the `geneticrowinds` argument. 

For simplicity, we have implemented a function `matchindices(meanformula, reformula, wsvarformula, idvar, df, geneticsamples)` which can be used to do this.
Input the mean, random effects, and within-subject variance formulas, the grouping (id) variable,
the dataframe (or table), and a vector of the IDs in the order of the genetic file and it returns `covrowmask, geneticrowmask` for matching indicies in a covariate file and geneticfile.

Note: the `idvar` in the dataframe and the `geneticsamples` vector must have the same element type. To convert a vector of integers to strings `string.(intvector)` can be used. To go from a vector of strings to integers you can use"  `map(x -> parse(Int, x), stringvectoparse)`.

In [8]:
famfileids = CSV.read(datadir * "hapmap3.fam", DataFrame, header = false)[!, 1] # famfile contains the sample IDs for PLINK files

324-element Array{String,1}:
 "A1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"
 "10"
 "11"
 "12"
 "13"
 ⋮
 "313"
 "314"
 "315"
 "316"
 "317"
 "318"
 "319"
 "320"
 "321"
 "322"
 "323"
 "Z324"

In [9]:
covdf = CSV.read(datadir * "vgwas_plinkex.csv", DataFrame)
first(covdf, 11)

,sex,onMeds,snp1,snp2,snp3,snp4,y,id
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,1.0,0.0,1.0,2.0,0.0,12.2667,A1
2,0.0,0.0,0.0,1.0,2.0,0.0,10.2681,A1
3,0.0,0.0,0.0,1.0,2.0,0.0,12.166,A1
4,0.0,0.0,0.0,1.0,2.0,0.0,11.8797,A1
5,0.0,0.0,0.0,1.0,2.0,0.0,12.8127,A1
6,0.0,0.0,0.0,1.0,2.0,0.0,9.98766,A1
7,0.0,0.0,0.0,1.0,2.0,0.0,12.1408,A1
8,0.0,1.0,0.0,1.0,2.0,0.0,13.2058,A1
9,0.0,0.0,0.0,1.0,2.0,0.0,11.3631,A1


In [10]:
covrowmask, geneticrowmask = matchindices(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id, covdf, famfileids);

In [11]:
vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        datadir * "hapmap3",
        pvalfile = pvalpath,
        nullfile = nullpath,
        covrowinds = covrowmask,
        geneticrowinds = geneticrowmask)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.341146
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.149456



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



In [12]:
# clean up
rm("vgwas.null.txt", force=true)
rm("vgwas.pval.txt", force=true)

## Plotting Results

To plot the GWAS results, we recommend using the [MendelPlots.jl package](https://openmendel.github.io/MendelPlots.jl/latest/).

## Multiple Plink file sets

In large scale studies, genotypes data are split into multiple Plink files, e.g., by chromosome. Then GWAS analysis can be done in parallel. This can be achieved by two steps.

Let's first create demo data by splitting hapmap3 according to chromosome:

In [61]:
# split example hapmap3 data according to chromosome
SnpArrays.split_plink(datadir * "hapmap3", :chromosome; prefix=datadir * "hapmap3.chr.")
readdir(glob"hapmap3.chr.*", datadir)

75-element Array{String,1}:
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.1.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.1.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.1.fam"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.10.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.10.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.10.fam"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.11.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.11.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.11.fam"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.12.bed"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.12.bim"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.12.fam"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.13.bed"
 ⋮
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.6.bed"
 "/User

Step 1: Fit the null model. Setting third argument `geneticfile` to `nothing` instructs `vgwas` function to fit the null model only.

In [62]:
nm = vgwas(@formula(y ~ 1 + sex + onMeds),
        @formula(y ~ 1),
        @formula(y ~ 1 + sex + onMeds),
        :id,
        datadir * "vgwas_plinkex.csv",
        nothing)

run = 1, ‖Δβ‖ = 0.037090, ‖Δτ‖ = 0.136339, ‖ΔL‖ = 0.005441, status = Optimal, time(s) = 0.177518
run = 2, ‖Δβ‖ = 0.000913, ‖Δτ‖ = 0.019810, ‖ΔL‖ = 0.001582, status = Optimal, time(s) = 0.141628



Within-subject variance estimation by robust regression (WiSER)

Mean Formula:
y ~ 1 + sex + onMeds
Random Effects Formula:
y ~ 1
Within-Subject Variance Formula:
y ~ 1 + sex + onMeds

Number of individuals/clusters: 324
Total observations: 3240

Fixed-effects parameters:
────────────────────────────────────────────────────────
                  Estimate  Std. Error       Z  Pr(>|Z|)
────────────────────────────────────────────────────────
β1: (Intercept)  13.2282     0.146459    90.32    <1e-99
β2: sex          -3.29295    0.2101     -15.67    <1e-54
β3: onMeds        0.459585   0.0596002    7.71    <1e-13
τ1: (Intercept)   0.792508   0.0850728    9.32    <1e-19
τ2: sex          -0.2865     0.0970732   -2.95    0.0032
τ3: onMeds        0.422303   0.063825     6.62    <1e-10
────────────────────────────────────────────────────────
Random effects covariance matrix Σγ:
 "γ1: (Intercept)"  3.32057



Step 2: GWAS for each chromosome.

In [64]:
# this part can be submitted as separate jobs
for chr in 1:23
    plinkfile = datadir * "hapmap3.chr." * string(chr)
    pvalfile = plinkfile * ".pval.txt" 
    vgwas(nm, plinkfile, pvalfile=pvalfile)
end

In [65]:
# show the result files
readdir(glob"*.pval.txt", datadir)

23-element Array{String,1}:
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.1.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.10.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.11.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.12.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.13.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.14.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.15.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.16.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.17.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.18.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.19.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.2.pval.txt"
 "/Users/christophergerman/.julia/dev/vGWAS/data/hapmap3.chr.20.pval.txt"
 "/Users/chr

In the rare situations where the multiple sets of Plink files lack the `fam` file or the corresponding bed and bim files have different filenames, users can explicitly supply bed filename, bim file name, and number of individuals. Replace Step 2 by 

Step 2': GWAS for each chromosome.

In [67]:
# this part can be submitted as separate jobs
for chr in 1:23
    bedfile = datadir * "hapmap3.chr." * string(chr) * ".bed"
    bimfile = datadir * "hapmap3.chr." * string(chr) * ".bim"
    pvalfile = datadir * "hapmap3.chr." * string(chr) * ".pval.txt"
    vgwas(nm, bedfile, bimfile, 324; pvalfile=pvalfile)
end

In [69]:
# clean up
rm("vgwas.null.txt", force=true)
isfile(datadir * "fittednullmodel.jld2") && rm(datadir * "fittednullmodel.jld2")
for chr in 1:23
    pvalfile = datadir * "hapmap3.chr." * string(chr) * ".pval.txt"
    rm(pvalfile, force=true)
end
for chr in 1:26
    plinkfile = datadir * "hapmap3.chr." * string(chr)
    rm(plinkfile * ".bed", force=true)
    rm(plinkfile * ".fam", force=true)
    rm(plinkfile * ".bim", force=true)
end

## Multiple Plink file sets on cluster

Running analyses on multiple plink file sets on a cluster computing system is demonstrated with our [OrdinalGWAS.jl pacakge](https://openmendel.github.io/OrdinalGWAS.jl/latest/#Multiple-Plink-file-sets-on-cluster). A similar setup can be used with vGWAS. 

## Troubleshooting

If there are issues you're encountering with running vGWAS, the following are possible remedies.

- Null Model
    - Issues with null model convergence may be solved by choosing alternate starting values for parameters, using a different solver, transforming variables, and increasing the number of runs (WiSER runs). These are detailed in the [WiSER documentation here](https://github.com/OpenMendel/WiSER.jl/blob/master/docs/src/model_fitting.md#tips-for-improving-estimation).
    
- GWAS Results
    - If you use the score test instead of the SPA-score test (SPA is default for single-SNP analyses), then there can be inflation in type I error and decreased power when (a) the sample size is small, (b) the number of repeated measures is low, or (c) the variants analyzed are rare with low minor allele frequencies. In these cases, the score test is not optimal and it is suggested to use the SPA version (`usespa=true`). SPA is only implemented for single-SNP analyses. These issues can occur in both Wald and score tests. 
    
If you notice any problems with your output or results, [file an issue](https://github.com/OpenMendel/vGWAS.jl/issues). 